# Multitask Learning for BERT Example
In this example RuBERT Base is trained on DaNetQA task of Russian SuperGLUE and sentiment analysis task on "mokoron" dataset of tweets

In [1]:
import transformers
from rutasks import rsg, mokoron
import datasets
from data_utils import dict_map
from multitask_transformers import MultitaskModel, make_features, MultitaskTrainer, NLPDataCollator, unpack_splits, Task

Tasks are created using Task(...) with `cls` - model class from `transformers` (Huggingface Transformers module), `config` from `transformers`, `data` dataset with parts from `datasets`(Huggingface Datasets module), `converter_to_features` being a function mapping one batch to defaultish features usually fed into models of `transformers` and an optional `name` for verbose messages

In [8]:
base_model_name = "DeepPavlov/rubert-base-cased"
tokenizer = transformers.AutoTokenizer.from_pretrained(base_model_name)
conv_params = {"pad_to_max_length": True, "max_length": 512}
tasks = {
    'danetqa': Task(
        cls=transformers.AutoModelForSequenceClassification,
        config=transformers.AutoConfig.from_pretrained(base_model_name, num_labels = 2),
        data=dict_map(datasets.load_dataset("russian_super_glue", 'danetqa'), rsg.preprocess_danetqa, name="danetqa"),
        converter_to_features=rsg.InputLabelConv(tokenizer, **conv_params),
        name=f"Russian SuperGLUE: DaNetQA"
    ),
    'mokoron': Task(
        cls=transformers.AutoModelForSequenceClassification,
        config=transformers.AutoConfig.from_pretrained(base_model_name, num_labels = 2),
        data=mokoron.load(),
        converter_to_features=rsg.InputLabelConv(tokenizer, input_name='text', **conv_params)
    )
}

loading configuration file https://huggingface.co/DeepPavlov/rubert-base-cased/resolve/main/config.json from cache at /Users/s1m00n/.cache/huggingface/transformers/a43261a78bd9edbbf43584c6b00aa94c032301840e532839cb5989362562a5d5.e8f15c5aad2f4653e46ceeba0bb32c02a629d106a902c964bce60523d290ac8f
Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_ve

  0%|          | 0/3 [00:00<?, ?it/s]

map(preprocess_danetqa, danetqa):
    map(preprocess_danetqa, danetqa[train])
    map(preprocess_danetqa, danetqa[validation])
    map(preprocess_danetqa, danetqa[test])


loading configuration file https://huggingface.co/DeepPavlov/rubert-base-cased/resolve/main/config.json from cache at /Users/s1m00n/.cache/huggingface/transformers/a43261a78bd9edbbf43584c6b00aa94c032301840e532839cb5989362562a5d5.e8f15c5aad2f4653e46ceeba0bb32c02a629d106a902c964bce60523d290ac8f
Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_ve

Making multitask features and unpack splits for each task

In [9]:
features = make_features(tasks)
train, validation = unpack_splits(features, "train", "validation")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/homebrew/Caskroom/miniforge/base/envs/data/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Truncation was not explicitly activated but `max_length` is pro

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/homebrew/Caskroom/miniforge/base/envs/data/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Truncation was not explicitly activated but `max_length` is pro

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/homebrew/Caskroom/miniforge/base/envs/data/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Truncation was not explicitly activated but `max_length` is pro

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/homebrew/Caskroom/miniforge/base/envs/data/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Truncation was not explicitly activated but `max_length` is pro

/opt/homebrew/Caskroom/miniforge/base/envs/data/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/data/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/homebrew/Caskroom/miniforge/base/envs/data/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Truncation was not explicitly activated but `max_length` is pro

In [10]:
train

{'danetqa': Dataset({
     features: ['attention_mask', 'input', 'input_ids', 'label', 'labels', 'token_type_ids'],
     num_rows: 1748
 }),
 'mokoron': Dataset({
     features: ['__index_level_0__', 'attention_mask', 'input_ids', 'label', 'labels', 'text', 'token_type_ids'],
     num_rows: 181467
 })}

In [11]:
validation

{'danetqa': Dataset({
     features: ['attention_mask', 'input', 'input_ids', 'label', 'labels', 'token_type_ids'],
     num_rows: 820
 }),
 'mokoron': Dataset({
     features: ['__index_level_0__', 'attention_mask', 'input_ids', 'label', 'labels', 'text', 'token_type_ids'],
     num_rows: 22683
 })}

In [12]:
model = MultitaskModel.create(base_model_name, tasks)
trainer = MultitaskTrainer(
    model=model,
    args=transformers.TrainingArguments(
        output_dir="test",
        overwrite_output_dir=True,
        learning_rate=1e-5,
        logging_steps=100,
        eval_steps=1000,
        do_train=True,
        num_train_epochs=1,
        per_device_train_batch_size=15,
        per_device_eval_batch_size=128, # TODO: this value is not read in MultitaskTrainer (strange bug)
        save_steps=5000,
    ),
    data_collator=NLPDataCollator(),
    train_dataset=train,
    eval_dataset=validation
)
trainer.train()

loading weights file https://huggingface.co/DeepPavlov/rubert-base-cased/resolve/main/pytorch_model.bin from cache at /Users/s1m00n/.cache/huggingface/transformers/1a4e163b2517b97a0eed0a06dc8ba7627019fed06fb41085ebcf0b777a07e862.3bce95e996437017822d238a7537c086a965b9843c96ad0d1d0f5a44c3faec75
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- T

KeyboardInterrupt: 